<a href="https://colab.research.google.com/github/nunocesarsa/CML_Short_tutorial/blob/master/Philip_AOI_Clipping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing google earth engine package

In [0]:
#importing Earth Engine package
import ee

#Authenticate with Google earth engine

In [2]:
#then we need to authenticate
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=vIBasNlW_j0rx_0eINcAoIpz8VFYuGOv5mP4xYRPRoM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/zAH55qZiXgOpAQmLDk-_I32iruygbWEsyOPN3fvGfby8CkciBNbv9nM

Successfully saved authorization token.


#Connecting to google drive

In [3]:
#and now we link the colab also to the google drive so we can use it to store the outputs
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#loading an image to visualize

In [30]:
#this is an area near portugal
poly_area_coord = [[[5.246113067692444,52.384667950129675],
                    [5.450046783512756,52.384667950129675],
                    [5.450046783512756,52.498933771499490],
                    [5.246113067692444,52.498933771499490],
                    [5.246113067692444,52.384667950129675]]]


#this is an area in the states
#poly_area_coord =[[[-80.03741893712676,32.74469942210935],
#                   [-79.8582044595877,32.74469942210935],
#                   [-79.8582044595877,32.88319873037949],
#                   [-80.03741893712676,32.88319873037949],
#                   [-80.03741893712676,32.74469942210935]]]

#This creates an earth engine polygon geometry
poly_area = ee.Geometry.Polygon(poly_area_coord)

#now we can use this, as a guide for selecting data
#check out the product table itself: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T2_SR
l8_collection = (ee.ImageCollection("COPERNICUS/S2_SR") #selects the L8 Surface reflectance product in GEE
                 .select(['B2','B3','B4',
                          'B5','B6','B7',
                          'B8A','B11','B12']) #selects the bands of interest - in this case we are actually picking up all of them
                 .filter(ee.Filter.date('2020-03-01','2020-03-31')) #filters on the time - you can play around with this to select different dates..
                 .filterBounds(poly_area) #filters on the Aoi we have created above
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)))



#l8_collection = (ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") #selects the L8 Surface reflectance product in GEE
#                 .select(['B1','B2','B3','B4',
#                          'B5','B6','B7',
#                          'B10','B11']) #selects the bands of interest - in this case we are actually picking up all of them
#                 .filter(ee.Filter.date('2017-06-01','2017-06-30')) #filtes on the time - you can play around with this to select different dates..
#                 .filterBounds(poly_area) #filters on the Aoi we have created above
#                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10))
#                 )

count = l8_collection.size()
print('Nr of images in collection: ', str(count.getInfo())+'\n')

Nr of images in collection:  2



In [0]:
mid_step = l8_collection.toList(count)
l8_img = ee.Image(mid_step.get(0))
l8_img_meta = l8_img.getInfo()
#l8_img_meta #uncomment here if you want to see the dump

#we can use ANY of these information on the metada to drive the query. For example, you might want to use a "row path" for landsat image instead of a polygon
#or perhaps you want to use only data of specific quality. Based on this metadata is where you would make this selection

#exploring the data

In [32]:
for i in range(count.getInfo()):
  mid_step = l8_collection.toList(count) #this is an internal step to help us loop on the data itself
  l8_img = ee.Image(mid_step.get(i)) #this fetches ith image on the collection 
  l8_img_meta = l8_img.getInfo()   #this creates an object which has only the metadata 
  print('At position',i,'is Product ID: ',l8_img_meta.get('properties',{}).get('PRODUCT_ID'))

  #interpreting the output:
  #Landsat naming convention:
  #XXXX_YYYYYY_ZZZZZZZZ
  #XXXX <- Product identifier LC08 is Landsat 8, level C
  #YYYYYYY <- Path and row identifers -> USGS pushes the LS images into a grid for the entire planet, this is the idenfier of said grid
  #ZZZZZZZZ <- Date in western calendar

At position 0 is Product ID:  S2B_MSIL2A_20200323T103639_N0214_R008_T31UFU_20200323T151634
At position 1 is Product ID:  S2B_MSIL2A_20200326T104639_N0214_R051_T31UFU_20200326T141923



# In the next part, we explore the image

For this, we import folium pacakge, and create a specific function to assist. This is to make a prettier and more explorable map than the normal.

(this was copy pasted, probably there are other ways of doing the same).

This tutorial gives an excellent run down example of the many options in folium

https://spatial.utk.edu/maps/ee-api-folium-setup.html

In [0]:
#in here, we select an image of the list - in python lists start at 0
mid_step = l8_collection.toList(count)
l8_img = ee.Image(mid_step.get(1))

#this command clips the image based on the original poly area
l8_img_clip = l8_img.clip(poly_area)

#first step now is to then generate a s2 image cropped to our aoi, and for information i will also plot it on the map

# Import the Folium library. This library is to assist in visualizations
import folium

# Import the Image function from the IPython.display module. 
from IPython.display import Image


#the next code snippet i stole from someone
# Define a method for displaying Earth Engine image tiles to folium map.

#this is a simplified version of the original function
def add_ee_layer(self, ee_image_object, vis_params, name):
  
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


# This first step sets up where is the "default starting position" of the map. And also the zoom
#my_map = folium.Map(location=[33,-80], zoom_start=8) #this had to be tried one by one... feels daft
my_map = folium.Map(location=[0,0], zoom_start=2) #this had to be tried one by one... feels daft


# Set visualization parameters. Here is we can create different visualizations of our data
visParams = {"bands": ['B4', 'B3', 'B2'],"gain": '0.1, 0.1, 0.1',"scale":20}
visParams_nir = {"bands": ['B8A', 'B4', 'B3'],"gain": '0.1, 0.1, 0.1',"scale":20}

# Add the image to the folium map
my_map.add_ee_layer(l8_img, visParams, 'RGB composite')
my_map.add_ee_layer(l8_img_clip, visParams_nir, 'NIR composite - clipped')
my_map.add_ee_layer(l8_img, visParams_nir, 'NIR composite')

#it is also possible to do calculations on the band - this saves a lot of computation time from your own computer

#normalized difference is already set up on google earth engine so it is especially easy to use
#ndvi = l8_img.normalizedDifference(['B8A', 'B4'])
#i copied this pallete from: https://developers.google.com/earth-engine/datasets/catalog/MODIS_MOD09GA_006_NDVI
#ndviParams = {'min': -1, 'max': 1, 'palette': ['lightblue','cyan','blue','darkblue','brown','lightgreen','green','darkgreen']};
#ndviParams = {'min': 0, 'max': 1, 'palette': ['blue','brown','white','lightgreen','green','darkgreen']}; #this is a bit trial and error actualy..

#now we just add it
#my_map.add_ee_layer(ndvi, ndviParams, 'NDVI')

#but if we want to make more complex operations, it is also possible
#and why not a normalized burn ratio, used for this purpose: https://en.wikipedia.org/wiki/Enhanced_vegetation_index
#evi = l8_img.expression('G*((Band5 - Band4)/(Band5+6*Band4-7.5*Band2+L))', 
#                        {'G':1,
#                         'L':2,
#                         'Band5':l8_img.select("B8A"),
#                         'Band4':l8_img.select("B4"),
#                         'Band2':l8_img.select("B3")}) #first we create an expression and then we tell it what is what. We can change the scalers and etc

#eviParams =  {'min': 0, 'max': 1, 'palette': ['blue','brown','white','lightgreen','green','darkgreen']}; #this is a bit trial and error actualy.. and the map will likely look horrible
#my_map.add_ee_layer(evi, eviParams, 'EVI')

#my_map.add_ee_layer(LC_proj,landcoverVis,"ola")


# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())


# Display the map.
display(my_map)

Now, here, read carefully. The way it works is the google colab will send a "task" order to GEE and that task will be running on the background.

This means that if you run the next box many times you will have multiple tasks being sent.

On the other hand, if you save a "big image" then you take a lot of time and space of your drive. So you can just extract the clipped area of your study location and focus on that - makes it easier to set up your own models on your own machine this way.

In [0]:
#getting the filenames
imagename = l8_img_meta.get('properties',{}).get('PRODUCT_ID')

In [68]:
l8_img_meta.get('properties',{}).get('PRODUCT_ID')

'S2B_MSIL2A_20200326T104639_N0214_R051_T31UFU_20200326T141923'

In [64]:
imagename[0:19]

'S2B_MSIL2A_20200326'

In [0]:
#these are task commands to be sent to GEE
task_config = {
    'image': l8_img,
    'fileFormat': 'GeoTIFF',
    'folder': 'OVP_AOI_Phillip',
    'fileNamePrefix': imagename[0:19],
    'description': "clipped area",
    'scale':20,
    'region':poly_area
}

#This is how we order it to start
task = ee.batch.Export.image.toDrive(**task_config )
task.start()

In [67]:
#to check if the task is running or completed, or failed, run this
task.status()

{'creation_timestamp_ms': 1587993934232,
 'description': 'clipped area',
 'id': 'JMW4HPYR5NWCATQXTNDHS3LK',
 'name': 'projects/earthengine-legacy/operations/JMW4HPYR5NWCATQXTNDHS3LK',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1587993934232}